## Bayesian Portfolio Construction and Error Minimization
##### Author: David Debreceni
##### Advisor: William Rubens, PhD

### Problem Statement
##### Can a portfolio constructed using a Bayesian process vs a Linear Regression produce a smaller estimate error?

### High Level Process
1. Using Fama French 3 Factor Model and Momentum build Linear regression over 20 years of data
2. Record the portfolio, weights, expected returns and actual returns for each period
3. Calculate the ERROR in the return
4. Run the process again for Bayesian Regression
5. Compare the ERROR estimates for each return period and report
 

Load Libraries for use in the analysis.  Many of the core methods are abstracted into a seperate Python file.

In [1]:
from ModelFunctionsBayes import getModelData as gmd
from ModelFunctionsBayes import utilityFuncs as utilF
from ModelFunctionsBayes import modelFuncs as MF
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import datetime
import random
from IPython.display import Math
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
import dill

output_notebook()


Loading BokehJS ...

Obtain the core data.  This will call the fama french website to obtain that data, and then use return data downloaded from Factset and stored in csv's for the analysis.
The default universe selected is the Russell 1000 constituents at monthly intervals.  Return data as far back as 20 years is captured for each security.
For speed of analysis purposes there is the ability to limit the universe selection via random sampling has been added.  The amount of history remains the same.  The random selection will happpen in the weights method to ensure there are datapoints to work with.

In [2]:
#get all the security data
dfSecRtn, dfFamaData, dfConstList, dfBmk = gmd().getAllData()
dfSecRtn.to_csv('output/dfSecRtn_v3.csv')
dfFamaData.to_csv('output/dfFamaData_v3.csv')
dfConstList.to_csv('output/dfConstList_v3.csv')
dfBmk.to_csv('output/dfBmk_v3.csv')
#set the random seed for reproducable results
random.seed( 30 )

The getWeights method is used for the Linear Regression part. 
Process:
1. for the date passed in get the current constituents as of that time
2. Limit the results to 100 records for special processing, these columns will be stored to use the same information for Bayes processing
3. Get the return, Fama French and RF data for processing
4. Run a linear regression for each security vs the Fama French data to get Beta's, Alpha (Intercept) and Error Estimates
5. Pass this information into covariance and expected return equation based upon the linear regression
6. Record the weights, expected return and columns used.

<center>Covariance Matrix<center>
$$Q = \beta \cdot \Sigma \cdot \beta^T   $$

<center>Expected Return<center>
$$R = \alpha +  \beta \cdot R_f   $$


In [3]:

def getWeights(curdt,dfSecRtn, dfFamaData,dfConstList, nextdt):

    #get the current securities as of this date
    colsTick = dfConstList[dfConstList[curdt.strftime('%Y-%m-%d')]==1].index.get_level_values(0).tolist()
    cols = dfSecRtn.loc[curdt,colsTick][~(dfSecRtn.loc[curdt,colsTick].isnull()) & ~(dfSecRtn.loc[:curdt,colsTick].isna().sum()>6)].index.tolist()
    #ensure there is not duplicate results as some entities have different tickers
    cols = list(set(cols))
    #Added for purposes of allowing for faster processing
    cols = random.sample(cols,100)
    
    #Get current subsets of data
    curSecRtn = dfSecRtn.loc[:curdt,cols]
    curFama= dfFamaData.loc[:curdt,['Mkt-RF','SMB','HML','Mom']]
    curRF= dfFamaData.loc[:curdt,'RF']    
    
    #get the current coef
    factBetas,factIntercepts, errEst = MF().buildFactorCoef(curSecRtn,curFama, curRF, cols)
        
    #predict weights based upon historical values
    Q = np.dot(np.dot(factBetas, curFama.cov()),factBetas.T) + np.diag(errEst) 
    
    #Expected return based upon Fama French
    R = (factIntercepts + np.dot(factBetas, curFama.iloc[-1])) + np.asscalar( curRF.iloc[-1])
    
    const = ({'type':'eq','fun':lambda x: np.sum(x)-1}
            ,{'type':'eq','fun':lambda x: np.sum(factBetas[:,0]*x)-1})  #constraint makes it BETA Neutral, ie should be the same as the market
    
    #allow a long only portfolio
    bounds = (0.0,1.0)
    
    #Build an optimal portfolio using a Mean Variance Utility equation
    OptWeights = MF() .weightOptimizer(MF().meanVar, len(R),const,bounds, R, Q, 0.5)
    
    #record the weights
    wghts = pd.DataFrame({'Entity':cols,'weight': list(np.round(OptWeights.x, 4))})
    #record the expected Return
    Ertn = np.nansum(wghts['weight']*R)
    
    return cols,  wghts, Ertn, R, Q, factBetas,factIntercepts, errEst

In [4]:
#Build dataframes for storing
dfEstLR = pd.DataFrame(columns=['Date','Rtn'])
dfActLR = pd.DataFrame(columns=['Date','Rtn'])
lstWeights = []
lstCols = []
lstReturnsLR = []
lstCovLR = []
lstfactBetasLR = []
lstfactInterceptsLR = []
lsterrEstLR = []

Start the data at a 10 year seed.  Build the history from there.  Currently the data constantly adds new information to the analysis.  Test for later is to make it a rolling window of history instead, say 10 years to see if the results are any closer.
This method will loop through getting the expected and actual return data for later analysis.

In [5]:
for i in np.arange(119,dfSecRtn.index.size-1): #
    
    curdt = dfSecRtn.index[i]
    nextdt = dfSecRtn.index[i+1]
    #caclulate the information bades upon historical values
    cols, wghts, ERtn, R, Q, factBetas,factIntercepts, errEst = getWeights(curdt,dfSecRtn, dfFamaData,dfConstList, nextdt)
    dfEstLR = dfEstLR.append({'Date':nextdt,'Rtn':ERtn}, ignore_index=True)
    #Calculate the actual portfolio return for next month with current weights
    nextMthRtn = dfSecRtn.loc[nextdt,cols]
    ActPort = np.nansum(wghts['weight']*np.asarray(nextMthRtn))
    dfActLR = dfActLR.append({'Date':nextdt ,'Rtn':ActPort}, ignore_index=True)
    lstWeights.append([nextdt,wghts])
    lstCols.append(cols)
    lstReturnsLR.append([curdt,R])
    lstCovLR.append([curdt,Q])
    lstfactBetasLR.append([curdt,factBetas])
    lstfactInterceptsLR.append([curdt,factIntercepts])
    lsterrEstLR.append([curdt,errEst])
    #LR Results
    dfEstLR.to_csv('output/dfEstLR_v3.csv')
    dfActLR.to_csv('output/dfActLR_v3.csv')
    #weight variables
    dill.dump([lstWeights,lstCols,lstReturnsLR,lstCovLR,lstfactBetasLR, lstfactInterceptsLR,lsterrEstLR,cols], open('output/FinalProject_v3_LRVariables.db','wb'))
print('Done')

Done


In [6]:
dfActLR.set_index('Date', inplace=True)
dfEstLR.set_index('Date', inplace=True)

In [7]:
#get the constituent counts
dfPortCount = pd.DataFrame([ (w[0], sum([np.round(c,2) > 0 for c in w[1]['weight']])) for w in lstWeights], columns=['Date','Count'])
#get the difference in returns this is the error we are looking to minimize
dfDifLR = (dfActLR - dfEstLR)

Show the count of the constituents for each month. Want to ensure that we have a diversified portfolio.  Hoping for a minimum of 20 securities of the 100 sample.
As you can see from the graph below, the portfolio selection is diverse with respect to the number of securities in each portfolio

In [8]:
p = figure(title="Constituent Counts", toolbar_location=None, x_axis_type='datetime')
p.vbar(x=dfPortCount.Date, top=dfPortCount.Count , width=.9)
show(p)

Below demonstrates the differences between the actual portfolio return and the estimated portfolio return.  There are periods where the values are very close, but for the most part they are quite far apart.See the Variance for the portfolio return.

In [9]:
p = figure(title="Difference between predicted return for period and actual return", toolbar_location=None, x_axis_type='datetime')
p.vbar(x=dfDifLR.index, top=dfDifLR.Rtn , width=1)
show(p)

As you can see from the graph below, a compounding portfolio built on the expectations including the error can have a dramatic impact to the long term results of a portfolio.  This is why reducing the error is so critical

In [10]:
#Return analysis between predicted return weights and Index
p = figure(title="Return Comparison", toolbar_location=None, x_axis_type='datetime')

p.line(x=dfBmk[dfEstLR.index.min():].index, y=np.cumprod(1+dfBmk[dfEstLR.index.min():].RETURN/100)*100, color='red', legend='R1000')
p.line(x=dfEstLR.index, y=np.cumprod(1+dfEstLR.Rtn/100)*100, color='green', legend='Expected Return')
p.line(x=dfActLR.index, y=np.cumprod(1+dfActLR.Rtn/100)*100, color='blue', legend='Actual Return')
p.legend.location = "top_left"
show(p)

Bayes

In [11]:
def getBayesWeights(curdt,dfSecRtn, dfFamaData,dfConstList, nextdt, cols):

    #get the current securities as of this date
    #colsTick = dfConstList[dfConstList[curdt.strftime('%Y-%m-%d')]==1].index.get_level_values(0).tolist()
    #cols = dfSecRtn.loc[curdt,colsTick][~(dfSecRtn.loc[curdt,colsTick].isnull()) & ~(dfSecRtn.loc[:curdt,colsTick].isna().sum()>6)].index.tolist()
    #ensure there is not duplicate results as some entities have different tickers
    #cols = list(set(cols))
    #Added for purposes of allowing for faster processing
    #cols = random.sample(cols,100)
    
    #Get current subsets of data
    curSecRtn = dfSecRtn.loc[:curdt,cols]
    curFama= dfFamaData.loc[:curdt,['Mkt-RF','SMB','HML','Mom']]
    curRF= dfFamaData.loc[:curdt,'RF']    
    
    #get the current coef
    factBetas,factIntercepts, errEst, factRtns = MF().buildBayesFactorCoef(curSecRtn,curFama, curRF, cols)
        
    #predict weights based upon historical values
    Q = np.dot(np.dot(factBetas, curFama.cov()),factBetas.T) + np.diag(errEst) 
    
    #Expected return based upon Fama French
    R = factRtns #(factIntercepts + np.dot(factBetas, curFama.iloc[-1])) + np.asscalar( curRF.iloc[-1])
        
    const = ({'type':'eq','fun':lambda x: np.sum(x)-1}
            ,{'type':'eq','fun':lambda x: np.sum(factBetas[:,0]*x)-1})  #constraint makes it BETA Neutral, ie should be the same as the market
    
    #allow a long only portfolio
    bounds = (0.0,1.0)
    
    #Build an optimal portfolio using a Mean Variance Utility equation
    OptWeights = MF() .weightOptimizer(MF().meanVar, len(R),const,bounds, R, Q, 0.5)
    
    #record the weights
    wghts = pd.DataFrame({'Entity':cols,'weight': list(np.round(OptWeights.x, 4))})
    #record the expected Return
    Ertn = np.nansum(wghts['weight']*R)
    
    return wghts, Ertn, R, Q, factBetas,factIntercepts, errEst

In [12]:
#Build dataframes for storing
dfEstB = pd.DataFrame(columns=['Date','Rtn'])
dfActB = pd.DataFrame(columns=['Date','Rtn'])
lstWeightsB = []
lstReturnsB = []
lstCovB = []
lstfactBetasB = []
lstfactInterceptsB = []
lsterrEstB = []

In [13]:
now = datetime.datetime.now()
print('Bayes Started ' + now.strftime("%m/%d/%Y, %H:%M:%S"))
lstcnt = 0
for i in np.arange(119,dfSecRtn.index.size-1): #
    now = datetime.datetime.now() 
    print(str(i) + ' ' + now.strftime("%m/%d/%Y, %H:%M:%S"))
    curdt = dfSecRtn.index[i]
    nextdt = dfSecRtn.index[i+1]
    #caclulate the information bades upon historical values
    wghts, ERtn, R, Q, factBetas,factIntercepts, errEst = getBayesWeights(curdt,dfSecRtn, dfFamaData,dfConstList, nextdt, lstCols[lstcnt])
    dfEstB = dfEstB.append({'Date':nextdt,'Rtn':ERtn}, ignore_index=True)
    #Calculate the actual portfolio return for next month with current weights
    nextMthRtn = dfSecRtn.loc[nextdt,cols]
    ActPort = np.nansum(wghts['weight']*np.asarray(nextMthRtn))
    dfActB = dfActB.append({'Date':nextdt ,'Rtn':ActPort}, ignore_index=True)
    lstWeightsB.append([nextdt,wghts])
    lstReturnsB.append([curdt,R])
    lstCovB.append([curdt,Q])
    lstfactBetasB.append([curdt,factBetas])
    lstfactInterceptsB.append([curdt,factIntercepts])
    lsterrEstB.append([curdt,errEst])
    #LR Results
    dfEstB.to_csv('output/dfEstB_v3.csv')
    dfActB.to_csv('output/dfActB_v3.csv')
    #weight variables
    dill.dump([lstWeightsB,lstReturnsB,lstCovB,lstfactBetasB, lstfactInterceptsB,lsterrEstB], open('output/FinalProject_v3_BayesVariables.db','wb'))
    lstcnt +=1
now = datetime.datetime.now()   
print('Bayes Done ' + now.strftime("%m/%d/%Y, %H:%M:%S"))

Bayes Started 12/02/2019, 15:34:54
119 12/02/2019, 15:34:54
120 12/02/2019, 15:59:00
121 12/02/2019, 16:22:16
122 12/02/2019, 16:45:25
123 12/02/2019, 17:09:27
124 12/02/2019, 17:34:36
125 12/02/2019, 18:01:04
126 12/02/2019, 18:27:09
127 12/02/2019, 18:53:34
128 12/02/2019, 19:21:27
129 12/02/2019, 19:48:20
130 12/02/2019, 20:15:01
131 12/02/2019, 20:41:31
132 12/02/2019, 21:07:54
133 12/02/2019, 21:35:42
134 12/02/2019, 22:02:04
135 12/02/2019, 22:30:03
136 12/02/2019, 22:58:05
137 12/02/2019, 23:24:21
138 12/02/2019, 23:53:44
139 12/03/2019, 00:22:23
140 12/03/2019, 00:48:17
141 12/03/2019, 01:14:17
142 12/03/2019, 01:39:11
143 12/03/2019, 02:04:49
144 12/03/2019, 02:29:31
145 12/03/2019, 02:54:06
146 12/03/2019, 03:19:05
147 12/03/2019, 03:44:20
148 12/03/2019, 04:08:49
149 12/03/2019, 04:31:49
150 12/03/2019, 04:54:54
151 12/03/2019, 05:17:58
152 12/03/2019, 05:41:43
153 12/03/2019, 06:05:00
154 12/03/2019, 06:28:09
155 12/03/2019, 06:51:20
156 12/03/2019, 07:15:14
157 12/03/2019,

In [14]:
dfActB.set_index('Date', inplace=True)
dfEstB.set_index('Date', inplace=True)

In [15]:
#get the constituent counts
dfPortCountB = pd.DataFrame([ (w[0], sum([np.round(c,2) > 0 for c in w[1]['weight']])) for w in lstWeightsB], columns=['Date','Count'])
#get the difference in returns this is the error we are looking to minimize
dfDifB = (dfActB - dfEstB)

In [16]:
p = figure(title="Constituent Counts", toolbar_location=None, x_axis_type='datetime')
p.vbar(x=dfPortCountB.Date, top=dfPortCountB.Count , width=.9)
show(p)

In [17]:
dfDifB.Rtn.std()**2

43.922126353159875

In [18]:
p = figure(title="Difference between predicted return for period and actual return", toolbar_location=None, x_axis_type='datetime')
p.vbar(x=dfDifB.index, top=dfDifB.Rtn , width=1)
show(p)

In [19]:
#Return analysis between predicted return weights and Index
p = figure(title="Bayes Return Comparison", toolbar_location=None, x_axis_type='datetime')

p.line(x=dfBmk[dfEstB.index.min():dfEstB.index.max()].index, y=np.cumprod(1+dfBmk[dfEstB.index.min():dfEstB.index.max()].RETURN/100)*100, color='red', legend='R1000')
p.line(x=dfEstB.index, y=np.cumprod(1+dfEstB.Rtn/100)*100, color='green', legend='Expected Return')
p.line(x=dfActB.index, y=np.cumprod(1+dfActB.Rtn/100)*100, color='blue', legend='Actual Return')
p.legend.location = "top_left"
show(p)